In [1]:
# landsat classifier
import ee
ee.Initialize()

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import scipy
import folium
import shapely
from IPython.display import Image
import geopandas as gpd
import rasterio
import pygeohydro
import datetime
from geemap.plot import geopandas_to_ee
import geemap.foliumap as geemap

In [3]:
meadows_df = gpd.read_parquet("./raw_data/ucdmeadows.parquet")

In [91]:
#study_area_huc10s = ['1804000905']
study_area_huc12s = ['180201210605']
#study_area_huc12s = ['180400090505']
wbd = pygeohydro.watershed.WBD('huc12')
data = wbd.byids(field='huc12', fids=study_area_huc12s)
huc10 = meadows_df['HUC12'].apply(lambda x: x[:10])
huc12 = meadows_df['HUC12']
meadows = meadows_df.loc[huc12.isin(study_area_huc12s)]
study_area_ee = geopandas_to_ee(data)
meadows_ee = geopandas_to_ee(meadows)

In [92]:
# load sentinel image (average of spring)
sentinel = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(study_area_ee) \
    .filterDate('2019-04-01', '2019-07-01') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20) \
    .median() \
    .clip(study_area_ee)

# add sentinel image to map
# Map = geemap.Map()
# Map.centerObject(study_area_ee)
# Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'sentinel')
# Map

In [93]:
# add ndvi, ndmi
sentinel = sentinel.addBands(sentinel.normalizedDifference(['B8', 'B4']).rename('NDVI'))
sentinel = sentinel.addBands(sentinel.normalizedDifference(['B8', 'B11']).rename('NDMI'))

# keep only bands of interest
sentinel = sentinel.select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B11', 'NDVI', 'NDMI'])
sentinel

In [94]:
# 10m slope
slope = ee.Terrain.slope(ee.Image('USGS/3DEP/10m'))
slope = slope.clip(study_area_ee)
sentinel = sentinel.addBands(slope.rename('slope'))

# dynamic world


dwCol = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
  .filterBounds(study_area_ee) \
  .filterDate('2019-04-01', '2019-07-01') \

dwImage = ee.Image(dwCol.median()).clip(study_area_ee)

CLASS_NAMES = [
    'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
    'shrub_and_scrub', 'built', 'bare', 'snow_and_ice']

VIS_PALETTE = [
    '419bdf', '397d49', '88b053', '7a87c6', 'e49635', 'dfc35a', 'c4281b',
    'a59b8f', 'b39fe1']


sentinel = sentinel.addBands(dwImage.select(CLASS_NAMES).rename(CLASS_NAMES))
sentinel

In [108]:
sample_size = 5000
meadow_points = sentinel.sample(
    region=meadows_ee,
    scale=10,
    numPixels=sample_size
)
non_meadow_points = sentinel.sample(
    region=study_area_ee.geometry().difference(meadows_ee),
    scale=10,
    numPixels=sample_size*2
)
meadow_points = meadow_points.map(lambda feature: feature.set('class', 1))
non_meadow_points = non_meadow_points.map(lambda feature: feature.set('class', 0))
sample = meadow_points.merge(non_meadow_points)

sample = sample.randomColumn()

split = 0.8; 
training = sample.filter(ee.Filter.lt('random', split));
validation = sample.filter(ee.Filter.gte('random', split));
# save CSV?

In [109]:
cart = ee.Classifier.smileCart().train(training, 'class', sentinel.bandNames())
classified = sentinel.classify(cart)
classified_reduced = classified.reduceNeighborhood(reducer=ee.Reducer.mean(), kernel=ee.Kernel.square(5))

In [97]:
Map = geemap.Map()
Map.centerObject(study_area_ee)
Map.addLayer(sentinel, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 'sentinel')
Map.addLayer(classified, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, 'classified')
Map.addLayer(classified_reduced, {'min': 0, 'max': 1, 'palette': ['red', 'green']}, 'classified reduced')
Map.addLayer(meadows_ee, {'color': 'black', 'opacity': 0.5}, 'meadows')
Map

In [110]:
# make accuracy assessment
# apply classifier to validation set
validation_classified = validation.classify(cart)
confusion_matrix = validation_classified.errorMatrix('class', 'classification')

# print accuracy
print('Validation overall accuracy: ', confusion_matrix.accuracy().getInfo())
print('Validation kappa: ', confusion_matrix.kappa().getInfo())
print('Validation producers accuracy: ', confusion_matrix.producersAccuracy().getInfo())
print('Validation consumers accuracy: ', confusion_matrix.consumersAccuracy().getInfo())


Validation overall accuracy:  0.8892904073587385
Validation kappa:  0.7506368819813934
